# **DIGITAL DESIGN FLOW**
### ***INTRODUCTION*** 

 Our *Mobius Chip's Digital Design Flow* aims to **lower the barrier between software-based design and the practical realities of hardware implementation;** by interweaving schematic design, simulation, physical implementation, and testing, it enables users to transition smoothly and intuitively between these two domains.

 The Mobius Chip, engineered for educational use and DIY projects, houses pre-sized fundamental analog blocks **— including differential pairs, current mirrors, inverters, and two-stage OTAs.** This design grants easier access to CMOS circuits, shifting the focus from the complexities of physical circuitry to the practical learning and use of these fundamental circuits. 
 
 A schematic of the Mobius Chip is shown below:
 <div align='center'>
    <img src="../circuit_data/schematic_images\MobiusChip_pinmap_schem_p1topleft.jpg" alt="Visualizing the Mobius Chip: Schematic Diagram Annotated with Pinmap" width="600"/>
    <figcaption><em><strong>Visualizing the Mobius Chip:</strong> Schematic Diagram Annotated with Pinmap.</em> </figcaption>
</div>

### ***SIMULATION PLATFORM***

Design and simulation of schematics are facilitated by ***LTspice, utilizing a custom Mobius Chip Library.*** This library comprises symbols representing all subcircuits embedded in the chip. It's crucial to comprehend the physical configuration of the chip for accurate digital bitstream generation.

The configuration of the Mobius Chip is achieved digitally via a sizeable switch matrix. This matrix can connect any of the 65 analog pins (out of 68, excluding 3 digital pins) to any of the ten available **buses** or nodes (not shown in schematic). The numbering of the pins is depicted in the previous schematic.

#### Buses are denoted as: ***BUS01, BUS02, ... , BUS10***

For effective parsing of the Spice netlist file by the bitstream generation program, it's crucial that all nodes intended for digital configuration on chip <u>***strictly follow the above bus labeling convention.***</u> This adherence ensures accurate translation into the bitstream.

# **CAD FLOW EXAMPLE**
The example circuit we will use is a **two-stage single-ended output** ***Miller-compensated Operational Transconductance Amplifier (OTA).*** The first stage consists of a NMOS differential input pair, actively loaded by a PMOS current mirror. The output of the first stage is connected to the gate of an actively loaded PMOS input device in a common-source configuration, which serves as the second stage of the OTA. 

The OTA is configured by connecting the highlighted blocks (left), using four buses as depicted in the assembled schematic (right) below:
 <div align='center'>
    <img src="../circuit_data/schematic_images\MobiusChip_pinmap_schem_motaHighlighted.jpg" alt="highlighted mota blocks within Mobius Chip" style="margin-right: 10px;" width="440" height="400"/>
    <img src="../circuit_data/schematic_images\mota_schem_bw.jpg" alt="mota circuit schematic" width="360" height="400"/>
    <figcaption><em><strong>Configuring the OTA:</strong> On the left, the subcircuits used in the OTA are highlighted. On the right, a schematic of the internally assembled OTA is shown. </em></figcaption>
</div>

### ***Creating the LTspice Schematic/Netlist:*** 
1. Create a new schematic in the ***same location as the custom Mobius Library***
2. From the custom library, instantiate the following symbols:

    * <p style='color:yellow;'><em>nmos_currentmirror_array</em></p>
    * <p style='color:magenta;'><em>dp_nmos_1x_a / dp_nmos_1x_b</em></p>
    * <p style='color:blue;'><em>pmos_currentmirror</em></p>
    * <p style='color:green;'><em>cs_pmos_4x_a</em></p>
3. Connect the circuit
4. ***Create the bus labels and attach them to the nodes***
5. Save a copy of the spice netlist (*view -> spice netlist*) as a .txt file in the ***netlist_catalog*** directory 

The bus numbers are arbitrary; any four of the ten buses can be used as long as the labels adhere to the labeling convention.

## ***BITSTREAM GENERATION***

Bitream generation is done through the use of the **`BitStream`** Class. An instance of the class represents a specific circuit, defined by its name. The class contains both public and private attributes and methods which are summarized in the table below:

| Name | Type | Description |
| :--- | :--- | :--- |
| **`circ_name`** | **Object Attribute** | Stores the name of the circuit associated with the instance of the class. |
| **`netlist_path`** | **Object Attribute** | Stores the path to the circuit's netlist file. |
| **`output_path`** | **Object Attribute** | Stores the path to the output bits file. |
| **`bitmatrix`** | **Object Attribute** | Stores the 65 output bits for each of the 10 buses (10 x 65). |
| **`bitstream`** | **Object Attribute** | Stores the flattened bitmatrix (1 x 650) to send to the ADALM2000. |
| **`active_buslist`** | **Object Attribute** | Stores the list of active buses in the circuit. |
| **`active_pinlists`** | **Object Attribute** | Stores a list of pin lists for each active bus in the circuit. |
| **`instance_netlist`** | **Object Attribute** | Stores the instance definition lines from the spice netlist file. |
| `_subckt_pindict` | Class Attribute | A private dictionary that maps subcircuit port order indices to output pin numbers. |
| `__init__(self, circ_name)` | Constructor | Initializes an instance of the `BitStream` Class with a specified circuit name. |
| `_readNetlist(self, nfp)` | Private Method | Reads the netlist file and returns the lines stored in instance_netlist. |
| `_getActiveBus(self, item)` | Private Method | Returns the number of the bus connected to an instance port defined in the netlist (0 for no connection). |
| `_getActivePin(self, key, pidx)` | Private Method | Returns the active pin number on a bus based on subcircuit name and port index. |
| `_rmDigitalPins(self, tlist)` | Private Method | Removes digital pins from the output bitstream lines. |
| **`manualInput(self)`** | **Public Method** | Allows the user to manually input active bus numbers and corresponding active pins. |
| **`netlistInput(self)`** | **Public Method** | Generates active bus numbers and corresponding active pins from a netlist file. |
| **`generateBits(self)`** | **Public Method** | Generates the output bits stored in bitstream and bitmatrix using the active bus numbers and active pins. |
| **`compareStream(self, cstream)`** | **Public Method** | Compares the generated bitstream attribute to a list passed in through 'cstream'. |
| **`writeBitFile(self)`** | **Public Method** | Saves the generated bits to an output file (each bus group is written on a new line). |







**NOTE:** For class usage, <span style='color:red;'><em>interact exclusively with object attributes and public methods</em></span>. Class attributes and private methods are detailed solely to provide a deeper understanding.



###  Class usage and bitstream generation with the Miller OTA example

Import the `BitStream` class

In [1]:
import sys

sys.path.append("../generation_modules/src")

from BitStream import BitStream

Initialize a `mota_bitstream` object of the `BitStream` class

In [2]:
circname = "mota_schem"
netlist_path = "../circuit_data/netlist_catalog/" + circname + "_nl.txt"
example_bitstream = BitStream()
example_bitstream.netlistInput(netlist_path)

In [3]:
from IPython.display import Markdown

x = ""
for line in example_bitstream.instance_netlist:
    s = (line[i] for i in range(len(line)))
    # print(' '.join(s))
    x += " ".join(s) + "<br>"

Markdown(
    f"""
{'*<u>stripped netlist containing only instance definition lines:</u>*'} 

{x}
"""
)


*<u>stripped netlist containing only instance definition lines:</u>* 

XX1 NC_01 NC_02 BUS03 BUS04 NC_03 NC_04 NC_05 nmos_currentmirror_array<br>XX2 NC_06 BUS01 BUS03 dp_nmos_1x_a<br>XX3 NC_07 BUS02 BUS03 dp_nmos_1x_b<br>XX4 BUS01 BUS02 pmos_currentmirror<br>XX5 BUS04 BUS02 cs_pmos_4x_a<br>


active buses/pins generated for the `mota_bitstream` object 

In [4]:
bus_table = ""
for i in range(len(example_bitstream.active_buslist)):
    pinlist = [str(item) for item in example_bitstream.active_pinlists[i]]
    pinlist = ", ".join(pinlist)
    bus_table += f"| {example_bitstream.active_buslist[i]} | {pinlist} |\n"
Markdown(
    f"""
*<u>{circname}:</u>*        
| bus number | active pins |
|:----------:|:-----------:|
{bus_table}
"""
)


*<u>mota_schem:</u>*        
| bus number | active pins |
|:----------:|:-----------:|
| 1 | 35, 62 |
| 2 | 40, 58, 61 |
| 3 | 21, 37, 38 |
| 4 | 22, 57 |



### Output bitstream definition &rarr; a *10 x 65* matrix, **S**. 
This matrix represents a bitstream where each entry **S**[i,j] is a binary bit. The matrix rows and columns correspond to buses and pins, respectively, in a hardware switch matrix. 

We represent the matrix **S** as a collection of bus vectors $\mathbf{b_{n}}$, for each bus number **n**, the associated bus vector $\mathbf{b_{n}}$ is defined as:

\begin{equation}
    \mathbf{b_{n}}[j] = 
\begin{cases}
    p_{_{68 - j}}, &  j \in \{0, 1, 2, \ldots, 63\} \\
    p_{_{1}}, &  j = 64 
\end{cases}
\end{equation}

This means that the *jth* entry of the bus vector $\mathbf{b_{n}}$ corresponds to pin $p_{_{k}}$. *note that j is defined such that the pin numbers did not change when the digital pins (2, 3, 4) were removed.*

The bitstream matrix **S** is then defined as:

\begin{equation}
    \mathbf{S[i]} = \mathbf{b_{n}}, \quad  n = 10 - i, \quad  i \in \{0, 1, 2, \ldots, 9\}
\end{equation}

This means that the *ith* row of **S** corresponds to the $\mathbf{b_{n}}$ bus vector where $n = 10 - i$, such that the bus numbers range from *BUS01* to *BUS10*. In this way, we map the binary matrix entries $s_{_{i,j}}$ to *1* of 65 total pins that can each connect to *1* of *10* total buses, where a *'1'* denotes a connection.





*generate, check, and save the* `mota_bitstream` output bits



In [5]:
bitstream = example_bitstream.generateBitstream()

In [6]:
compare_filepath = "../circuit_data/bitstream_catalog/{}_bs.txt".format(circname)
example_bitstream.compareFile(compare_filepath)

files match


True

In [7]:
output_path = "../circuit_data/bitstream_catalog/" + circname + "_bs.txt"
example_bitstream.writeBitFile(output_path)

### **Output bitmatrix table**

In [8]:
import pandas as pd

bitmatrix = example_bitstream.bitmatrix
bitmatrix_df = pd.DataFrame(bitmatrix)
bitmatrix_df.columns = ["p" + str(68 - i) for i in range(65)]
bitmatrix_df.index = ["BUS0" + str(10 - i) for i in range(10)]
bitmatrix_df.columns.values[64] = "p1"
bitmatrix_df.index.values[0] = "BUS10"

pd.set_option("display.max_columns", None)
display(bitmatrix_df)

,p68,p67,p66,p65,p64,p63,p62,p61,p60,p59,p58,p57,p56,p55,p54,p53,p52,p51,p50,p49,p48,p47,p46,p45,p44,p43,p42,p41,p40,p39,p38,p37,p36,p35,p34,p33,p32,p31,p30,p29,p28,p27,p26,p25,p24,p23,p22,p21,p20,p19,p18,p17,p16,p15,p14,p13,p12,p11,p10,p9,p8,p7,p6,p5,p1
BUS10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS04,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS02,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
BUS01,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
print(
    "bitstream attribute: type: {}[{}]\nbitstream attribute: length: {}".format(
        type(example_bitstream.bitstream).__name__,
        type(example_bitstream.bitstream[0]).__name__,
        len(example_bitstream.bitstream),
    )
)

bitstream attribute: type: list[int]
bitstream attribute: length: 650
